In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# TODO:解释参数含义，在?处填入合适的参数
batch_size = 64 #batch size 对于gpu一般是2的倍数这里选择64,32应该也行
learning_rate = 0.001 # 一般的学习率都定为0.001,如果结果出来不好再进行调整
num_epochs = 10 #进行10轮的训练
 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))#这里加了一个数据归一化
])
 
# root可以换为你自己的路径
trainset = torchvision.datasets.CIFAR10(root='D:/Dataset/CIFAR-10', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
 
testset = torchvision.datasets.CIFAR10(root='D:/Dataset/CIFAR-10', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

#以上是利用torchvision下载数据集以及加载数据集

#检测一下数据，没问题就加注释了
#trainset_size=len(trainset)
#testset_size=len(testset)
#print(f'{trainset_size}')
#print(f'{testset_size}')

#利用卷积神经网络
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # TODO:这里补全你的网络层
        self.conv1=nn.Conv2d(3,16,3,padding=1)#三个参数分别为输入通道数，输出通道数和卷积核的尺寸，padding让结果和初始输入的尺寸相同
        self.bn1 = nn.BatchNorm2d(16)#进行批归一化，随后几步增加深度和复杂度
        self.conv2=nn.Conv2d(16,32,3,padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3=nn.Conv2d(32,64,3,padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4=nn.Conv2d(64,128,3,padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool=nn.MaxPool2d(2,2)#进行最大池化，减小大小
        self.fc1=nn.Linear(128*2*2,512)#三个全连接层
        self.fc2=nn.Linear(512,256)
        self.fc3=nn.Linear(256,10)
        
        self.dropout = nn.Dropout(0.5)#抛弃一些神经元，防止过拟合
    def forward(self, x):
        # TODO:这里补全你的前向传播
        x=self.pool(F.relu(self.bn1(self.conv1(x))))
        x=self.pool(F.relu(self.bn2(self.conv2(x))))
        x=self.pool(F.relu(self.bn3(self.conv3(x))))
        x=self.pool(F.relu(self.bn4(self.conv4(x))))
        x=x.view(-1,128*2*2)
        x=F.relu(self.fc1(x))
        x=self.dropout(x)
        x=F.relu(self.fc2(x))
        x=self.dropout(x)
        x=self.fc3(x)
        return x
        
 
# TODO:补全
model = Network().to(device)
 
criterion = nn.CrossEntropyLoss()#为分类问题，使用交叉熵损失函数
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
 
def train():
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
 
            optimizer.zero_grad()
 
            outputs = model(inputs)
            loss = criterion(outputs, labels)
 
            loss.backward()
            optimizer.step()
 
            running_loss += loss.item()
 
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
 
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')
 
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
 
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the 10000 test images: {accuracy:.2f}%')
 
if __name__ == "__main__":
    train()
    test()

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1.4372, Accuracy: 47.11%
Epoch [2/10], Loss: 1.0836, Accuracy: 61.91%
Epoch [3/10], Loss: 0.9230, Accuracy: 68.03%
Epoch [4/10], Loss: 0.8268, Accuracy: 71.82%
Epoch [5/10], Loss: 0.7393, Accuracy: 74.87%
Epoch [6/10], Loss: 0.6795, Accuracy: 77.01%
Epoch [7/10], Loss: 0.6174, Accuracy: 79.04%
Epoch [8/10], Loss: 0.5689, Accuracy: 80.49%
Epoch [9/10], Loss: 0.5317, Accuracy: 82.02%
Epoch [10/10], Loss: 0.4896, Accuracy: 83.38%
Accuracy of the model on the 10000 test images: 76.30%
